In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
import time
import json

# Ustawienie urządzenia: GPU, jeśli jest dostępne, w przeciwnym razie CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Dataset korzystający jedynie z obrazków
class ImageOnlyDataset(Dataset):
    def __init__(self, json_file, image_dir, transform=None):
        self.data = pd.read_json(json_file, lines=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx]['img_local'])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        price = self.data.iloc[idx]['price']
        return {
            'image': image,
            'price': torch.tensor(price, dtype=torch.float32)
        }

# Model korzystający jedynie z obrazków (CNN)
class ImageOnlyModel(nn.Module):
    def __init__(self):
        super(ImageOnlyModel, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        num_features = self.cnn.fc.in_features
        self.cnn.fc = nn.Linear(num_features, 1)  # Modyfikacja ostatniej warstwy

    def forward(self, image):
        return self.cnn(image)

# Przygotowanie transformacji obrazu
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dane treningowe i testowe
image_dir = 'data_img'
train_json = 'train_data_with_car_type.json'
test_json = 'test_data_with_car_type.json'

train_dataset = ImageOnlyDataset(json_file=train_json, image_dir=image_dir, transform=transform)
test_dataset = ImageOnlyDataset(json_file=test_json, image_dir=image_dir, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Inicjalizacja modelu
model = ImageOnlyModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Trenowanie modelu
num_epochs = 30
for epoch in range(num_epochs):
    start_time = time.time()

    model.train()
    running_loss = 0.0
    for batch in train_dataloader:
        images = batch['image'].to(device)
        prices = batch['price'].to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, prices)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_dataloader)
    epoch_duration = time.time() - start_time
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Duration: {epoch_duration:.2f} seconds")

# Ewaluacja modelu
def evaluate_image_model(model, dataloader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in dataloader:
            images = batch['image'].to(device)
            prices = batch['price'].to(device)
            
            outputs = model(images).squeeze()
            loss = criterion(outputs, prices)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Image Model Test Loss (MSE): {avg_loss:.4f}")
    return avg_loss




In [ ]:
evaluate_image_model(model, test_dataloader)